# SAWI maps

This notebook generates the cartographic representation of the SAWI estimates for a territory.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from mapclassify import NaturalBreaks
import numpy as np
from shapely.geometry.point import Point
from matplotlib_scalebar.scalebar import ScaleBar
from geo_northarrow import add_north_arrow

In [ ]:
# Import the names of the territories
l_names = pd.read_csv('../data/input/table/mpios_names.txt',header=None)
l_names = list(l_names[0])

# Methods
methods = ['EQU','AHP','PCA']

In [ ]:
m = l_names[0]
for me in methods:
    k = 5
    blocks = gpd.read_file(f'../data/output/shape/blocks/{m}_blocks.shp')
    SAWI = gpd.read_file(f'../data/output/shape/accessibility/indexes/SAWI/SAWI_{m}_norm.shp')

    fig, ax = plt.subplots(figsize=(15, 10))
    blocks.plot(ax=ax, color='lightgray')

    # Create a column that marks values equal to 0
    SAWI[f'SAWI_{me}_cat'] = SAWI[f'SAWI_{me}'].apply(lambda x: 0 if x == 0 else 'not_0')

    # Classify non-zero values using natural_breaks
    not_0_values = SAWI.loc[SAWI[f'SAWI_{me}'] != 0, f'SAWI_{me}']
    classifier = NaturalBreaks(not_0_values, k=k-1)
    SAWI.loc[SAWI[f'SAWI_{me}'] != 0, f'SAWI_{me}_cat'] = classifier.yb + 1  # +1 para evitar conflictos con la categoría '0'

    # Define the color scheme including the new category
    colors = plt.cm.viridis_r(np.linspace(0, 1, k)).tolist()

    # If there are no zeros, create a tiny geometry to allow for plotting
    if len(not_0_values) == len(SAWI):
        last_register_index = len(SAWI)-1
        buff_around_first_register = SAWI.to_crs('epsg:32618').iloc[[0]].centroid.buffer(0.01).to_crs('epsg:4326').loc[0]
        SAWI.loc[last_register_index+1,f'SAWI_{me}_cat'] = 0
        SAWI.loc[last_register_index+1,'geometry'] = buff_around_first_register

    # Plot
    SAWI.plot(column=f'SAWI_{me}_cat', legend=True, cmap=plt.cm.colors.ListedColormap(colors), ax=ax, 
                    legend_kwds={'loc': 'lower right', 'title': f'SAWI {me}'})

    legend = ax.get_legend()
    original_legend_handles = legend.legend_handles
    original_legend_labels = ['0.00, 0.00'] + [f'0.00, {classifier.bins[0]:.2f}'] + [f'{classifier.bins[i]:.2f}, {classifier.bins[i+1]:.2f}' for i in range(len(classifier.bins)-1)]

    # Adjust the legend to include the new interval
    ax.legend(original_legend_handles, original_legend_labels, loc='lower right', title=f'SAWI {me}', title_fontsize='x-large', prop={'size': 'x-large'})
    # Scale for epsg:4326
    points = gpd.GeoSeries([Point(-73.5, 40.5), Point(-74.5, 40.5)], crs=4326)  # Geographic WGS 84 - degrees
    points = points.to_crs(32618)
    distance_meters = points[0].distance(points[1])
    scale = ScaleBar(dx=distance_meters,location='lower left',font_properties={'size':'x-large'})
    ax.add_artist(scale)
    # North arrow
    add_north_arrow(ax, scale=.4, xlim_pos=.1, ylim_pos=.9, color='#000', text_scaler=3, text_yT=-1.25)
    plt.axis('off')
    plt.tight_layout()
    # plt.show()
    plt.savefig(f'../data/output/image/SAWI/{m}_SAWI_{me}.png',dpi=300)
    plt.close()